In [8]:
#Reading Names.dmp and then Nodes.dmp files.
#Combining two files based on their Tax_id's to obtain a final file that is loaded into Neo4j
import re
import csv
import pandas as pd
dmp_file = open(r'C:\Users\GURU\Desktop\tax_data 26-3\names.dmp',"r", encoding="utf-8").read()
#dmp_file=dmp_file[0:4000]
dmp_file = re.sub('\s+', '', dmp_file) #Removing \t from file
dmp_file=re.sub(',','',dmp_file)       #Removing , from file as it causes issues
dmp_file= dmp_file.replace('|', ',')   #Using , in place of "|"
dmp_file=dmp_file.split(',')


In [9]:
def file_preprocess(file):
    count=0
    runcount=0
    result={}
    splitted_index_list=[]
    splitted_value_list=[]
 
    for index,item in enumerate(file):
       
        if index%4==0:
            result[index+1]=file[index]
        else:
            result[index+1]=file[index]
    return result

preprocess_output=file_preprocess(dmp_file)

        
        
   

In [10]:
result_dict={}
itercount=0
for index,item in enumerate(preprocess_output):
    if index==1:
        result_dict[itercount]=preprocess_output[index]+','+preprocess_output[index+1]+','+preprocess_output[index+2]+','+preprocess_output[index+3]
       
        itercount+=1
    if index%4==0: 
        result_dict[itercount]=preprocess_output[index+1]+','+preprocess_output[index+2]+','+preprocess_output[index+3]+','+preprocess_output[index+4]
        
        itercount+=1
    if index==len(preprocess_output)-4:
        result_dict[itercount]=preprocess_output[index+1]+','+preprocess_output[index+2]+','+preprocess_output[index+3]+','+preprocess_output[index+4]
       
        itercount+=1
        break
result_dict.pop(0)
result_dict.popitem()
        

(3238651, 'Ageneiogarra,,scientificname,')

In [11]:
#Get only scientific names and genbankcommonname
count=0
scientific_dict={}
common_dict={}
for k, v in list(result_dict.items()):
    if 'scientificname' in v:
        scientific_dict[count]=v
        count+=1
    else:
        if 'genbankcommonname' in v:
            common_dict[count]=v
            count+=1
          

In [12]:
#Get only 1,2 column only as 4th column is not needed all are redundant 'scientific name'
scientific_dict_2={}
for k, v in scientific_dict.items():
    scientific_dict_2[v.split(',')[0]]=v.split(',')[1]
common_dict_2={}
for k, v in common_dict.items():
    common_dict_2[v.split(',')[0]]=v.split(',')[1]
               
   

In [13]:
#merge commondict 2 and sc dict 2 into dataframe and merge
scientific_df = pd.DataFrame(list(scientific_dict_2.items()),columns=['tax_id','scientific_name'])
common_df = pd.DataFrame(list(common_dict_2.items()),columns=['tax_id','common_name'])
final_df=pd.merge(scientific_df, common_df,how='left',left_on='tax_id',right_on='tax_id')
final_df.common_name = final_df.common_name.fillna('NA')

In [14]:
#Dataframe to csv conversion
final_df.to_csv (r'C:\Users\GURU\Desktop\tax_data 26-3\name.csv', index = False, header=True)

###############################################################################################################

In [15]:
###############################################################################################################
#Nodes.dmp file conversion

dmp_file = open(r'C:\Users\GURU\Desktop\tax_data 26-3\nodes.dmp',"r", encoding="utf-8").read()
#dmp_file=dmp_file[0:1000]
dmp_file = re.sub('\s+', '', dmp_file)
dmp_file = dmp_file.replace('|', ',')
dmp_file=dmp_file.split(',')


In [16]:
def file_preprocess(file):
    count=0
    result={}
    splitted_index_list=[]
    splitted_value_list=[]
 
    for index,item in enumerate(file):
       
        if index%18==0:
            result[index+1]=file[index]
        else:
            result[index+1]=file[index]
    return result

In [17]:
preprocess_output=file_preprocess(dmp_file)

In [18]:
def add_item(i,item):
    item_list=""
    if i==1:
        for j in range(0,18):
            item_list+=item[i+j]+','
    if i%18==0:
         for j in range(1,18):
            item_list+=item[i+j]+','
        
    
    return item_list
        

In [19]:
result_dict={}
itercount=0
for index,item in enumerate(preprocess_output):
    if index==1:
        result_dict[itercount]=add_item(index,preprocess_output)
       
        itercount+=1
    if index%18==0: 
        result_dict[itercount]=add_item(index,preprocess_output)
        
        itercount+=1
    if index==len(preprocess_output)-18:
        result_dict[itercount]=add_item(index,preprocess_output)
       
        itercount+=1
        break
result_dict.pop(0)
result_dict.popitem()

(2323131, '')

In [20]:
#To get the first 3 columns

for k, v in result_dict.items():
    result_dict[k]=v.split(',')[0]+','+v.split(',')[1]+','+v.split(',')[2]
   


In [21]:
#dict to csv file
import csv
my_dict = result_dict
with open(r'C:\Users\GURU\Desktop\tax_data 26-3\test_node.csv', 'w') as f:
    for key in my_dict.keys():
        f.write("%s\n"%(my_dict[key]))

csv_file = pd.read_csv(r'C:\Users\GURU\Desktop\tax_data 26-3\test_node.csv', 
                  sep=',', 
                  names=["tax_id","parent tax_id","rank"])


csv_file.to_csv(r'C:\Users\GURU\Desktop\tax_data 26-3\node.csv', encoding='utf-8', index=False)

In [22]:
# Read the files into two dataframes. First add parent rank to the existing node.csv
df1 = pd.read_csv(r'C:\Users\GURU\Desktop\tax_data 26-3\node.csv')
df2 = pd.read_csv(r'C:\Users\GURU\Desktop\tax_data 26-3\name.csv')

In [29]:
# Merge the two dataframes,of (taxid,parentid,rank) and (taxid,scientificname,commonname) 
#using _ID column as key
df3 = pd.merge(df1, df2, on = 'tax_id')
#Modify first row
df3.at[0,'parent tax_id']='0'
df3.at[0,'rank']='norank'
df3.common_name = df3.common_name.fillna('NA')

# Write it to a new CSV file
df3.to_csv(r'C:\Users\GURU\Desktop\tax_data 26-3\Final.csv',encoding='utf-8', index=False)


